In [ ]:
sos run /home/rf2872/codes/xqtl-pipeline/pipeline/wasp_vcf.ipynb reformat_vcf_wasp \
    --QCed_VCF_files `ls /home/rf2872/Work/leaf_cutter2/db/rosmap_wgs/QC_ed/*filtered.vcf.gz` \  #maybe I should give real path 
    -J 50 -c ~/test/csg.yml -q csg


In [ ]:
#work
sos run /home/rf2872/codes/xqtl-pipeline/pipeline/wasp_vcf.ipynb reformat_vcf_wasp \
    --QCed_VCF_files `ls ~/Work/leaf_cutter2/db/rosmap_wgs/t/*gz` \
    -J 50 -c ~/test/csg.yml -q csg


In [ ]:
[global]
import os
# Work directory & output directory
parameter: cwd = path('./')
# The filename prefix for output data
parameter: job_size = 1
parameter: mem = '60G'
parameter: container = ''
parameter: QCed_VCF_files = paths
import pandas as pd
#parameter: analysis_units = path
# handle N = per_chunk data-set in one job
parameter: per_chunk = 1


In [ ]:
[reformat_vcf_wasp_1]
parameter: frequency_threshold = 0.01
input: QCed_VCF_files, group_by = 1
output:f'{cwd:a}/wasp_vcf/{_input:bnn}.AF.WASP.vcf.gz'
task: trunk_workers = 1, trunk_size = job_size, mem = mem,  walltime = '24h', tags = f'{step_name}_{_output[0]:bn}'
bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    cd $[cwd]
    module load BCFTOOLS/1.17
    #echo $[_input]
    #echo $[_output:nnn].vcf.gz 
    bcftools view -i "AF>$[frequency_threshold]" -Oz -o $[_output:nnn].vcf.gz  $[_input] 
    #bcftools view $[_output:nnn].vcf.gz | awk -v OFS='\t' '{if ($0 ~ /^##/) print $0; else {for (i=1; i<=9; i++) printf $i "\t"; printf "0/1\n"}}'| gzip > $[_output]
    sample_name=`bcftools query -l $[_output:nnn].vcf.gz | head -n1`
    bcftools view -s $sample_name $[_output:nnn].vcf.gz | awk -v OFS='\t' 'BEGIN {FS=OFS} !/^#/ && NR>1 {$9="GT"; $10="0/1"}1' | bcftools reheader -s <(echo "GENO") - | gzip -c > $[_output]


In [ ]:
[reformat_vcf_wasp_2]
input: group_by = 'all'
parameter: prefix = "ZOD14598_AD_GRM_WGS_2021-04-29_all"
parameter: suffix = "recalibrated_variants.leftnorm.filtered.AF.WASP.vcf.gz"
output: f'{cwd:a}/wasp_vcf/{prefix}.{suffix}'
task: trunk_workers = 1, trunk_size = job_size, mem = mem,  walltime = '24h', tags = f'{step_name}_{_output[0]:bn}'
bash: expand= "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    module load BCFTOOLS/1.17
    bcftools concat $[_input]  -Oz -o $[_output] 
